In [ ]:
!pip install yfinance


In [ ]:
!pip install "numpy<2" --force-reinstall


In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# -----------------------------------
# List of Top 10 Indian Stocks
# -----------------------------------
stocks = [
    "RELIANCE.NS", "HDFCBANK.NS", "TCS.NS", "INFY.NS",
    "ICICIBANK.NS", "SBIN.NS", "BHARTIARTL.NS",
    "BAJFINANCE.NS", "HINDUNILVR.NS", "LT.NS"
]

for ticker in stocks:
    print(f"\nTraining model for {ticker}")

    # -----------------------------------
    # 1. Download Data
    # -----------------------------------
    data = yf.download(ticker, start="2015-01-01")

    if data.empty:
        print(f"⚠ No data for {ticker}")
        continue

    # -----------------------------------
    # 2. Feature Engineering
    # -----------------------------------
    data['Return'] = data['Close'].pct_change()
    data['MA20'] = data['Close'].rolling(20).mean()
    data['MA50'] = data['Close'].rolling(50).mean()
    data['Volatility'] = data['Return'].rolling(20).std()
    data['Volume_Change'] = data['Volume'].pct_change()
    data['MA_Cross'] = (data['MA20'] > data['MA50']).astype(int)



    # RSI Calculation
    delta = data['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # -----------------------------------
    # 3. Target Creation (Future 5-day return)
    # -----------------------------------
    data['Future_Return'] = data['Close'].shift(-5) / data['Close'] - 1

    data['Signal'] = np.where(data['Future_Return'] > 0.01, 1,
                       np.where(data['Future_Return'] < -0.01, -1, 0))

    # -----------------------------------
    # 4. Drop Missing Values
    # -----------------------------------
   

    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.dropna()


    # -----------------------------------
    # 5. Select Features
    # -----------------------------------
    features = ['Return', 'MA20', 'MA50', 'Volatility', 'Volume_Change', 'RSI','MA_Cross']

    X = data[features]
    y = data['Signal']

    # -----------------------------------
    # 6. Time Series Train-Test Split
    # -----------------------------------
    split = int(len(data) * 0.8)

    X_train = X[:split]
    X_test = X[split:]
    y_train = y[:split]
    y_test = y[split:]

    # -----------------------------------
    # 7. Scaling
    # -----------------------------------
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # -----------------------------------
    # 8. Model Training
    # -----------------------------------
    model = RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        random_state=42
    )

    model.fit(X_train_scaled, y_train)

    # -----------------------------------
    # 9. Evaluation
    # -----------------------------------
    predictions = model.predict(X_test_scaled)
    print(classification_report(y_test, predictions))
    
    # -----------------------------------
    # 10. Save Model + Scaler + Features
    # -----------------------------------
    pickle.dump(model, open(f"{ticker}_model.pkl", "wb"))
    pickle.dump(scaler, open(f"{ticker}_scaler.pkl", "wb"))
    pickle.dump(features, open(f"{ticker}_features.pkl", "wb"))

    print(f"✅ {ticker} model saved successfully!")

print("\n🎉 All models trained and saved!")


Training model for RELIANCE.NS


[*********************100%***********************]  1 of 1 completed


              precision    recall  f1-score   support

          -1       0.31      0.75      0.44       173
           0       0.23      0.07      0.11       160
           1       0.28      0.09      0.13       205

    accuracy                           0.30       538
   macro avg       0.27      0.30      0.23       538
weighted avg       0.27      0.30      0.23       538

✅ RELIANCE.NS model saved successfully!

Training model for HDFCBANK.NS


[*********************100%***********************]  1 of 1 completed


              precision    recall  f1-score   support

          -1       0.28      0.76      0.41       137
           0       0.41      0.25      0.31       211
           1       0.51      0.12      0.19       190

    accuracy                           0.33       538
   macro avg       0.40      0.38      0.30       538
weighted avg       0.41      0.33      0.29       538

✅ HDFCBANK.NS model saved successfully!

Training model for TCS.NS


...

              precision    recall  f1-score   support

          -1       0.37      0.81      0.51       201
           0       0.31      0.03      0.06       120
           1       0.38      0.16      0.22       217

    accuracy                           0.37       538
   macro avg       0.35      0.33      0.26       538
weighted avg       0.36      0.37      0.29       538

✅ BAJFINANCE.NS model saved successfully!

Training model for HINDUNILVR.NS


[*********************100%***********************]  1 of 1 completed


              precision    recall  f1-score   support

          -1       0.39      0.21      0.27       195
           0       0.38      0.47      0.42       173
           1       0.38      0.49      0.43       170

    accuracy                           0.38       538
   macro avg       0.38      0.39      0.37       538
weighted avg       0.39      0.38      0.37       538

✅ HINDUNILVR.NS model saved successfully!

Training model for LT.NS


[*********************100%***********************]  1 of 1 completed


              precision    recall  f1-score   support

          -1       0.36      0.05      0.09       174
           0       0.28      0.54      0.37       145
           1       0.39      0.42      0.40       219

    accuracy                           0.33       538
   macro avg       0.34      0.34      0.29       538
weighted avg       0.35      0.33      0.29       538

✅ LT.NS model saved successfully!

🎉 All models trained and saved!
